# Week 10: Computer Vision & Image Processing

## Learning Objectives

By the end of this notebook, you will be able to:

- Understand the fundamentals of digital images and computer vision
- Perform basic image processing operations 
- Apply traditional computer vision techniques
- Build image classification models using neural networks
- Understand convolutional neural networks (CNNs)
- Work with real image datasets
- Apply computer vision to practical problems
- Understand modern computer vision applications

---

## 1. Introduction to Computer Vision

Computer Vision is a field of AI that enables computers to interpret and understand visual information from the world. Applications include:

- **Autonomous Vehicles**: Object detection, lane recognition
- **Healthcare**: Medical image analysis, disease diagnosis
- **Security**: Face recognition, surveillance systems
- **Retail**: Product recognition, visual search
- **Social Media**: Photo tagging, content moderation
- **Manufacturing**: Quality control, defect detection
- **Agriculture**: Crop monitoring, pest detection

In [ ]:
# Import essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# Computer vision libraries
try:
    import cv2
    CV2_AVAILABLE = True
    print("OpenCV available for advanced image processing!")
except ImportError:
    print("OpenCV not available. Install with: pip install opencv-python")
    CV2_AVAILABLE = False

# Deep learning libraries (optional)
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    TF_AVAILABLE = True
    print(f"TensorFlow available for deep learning! Version: {tf.__version__}")
    tf.random.set_seed(42)
except ImportError:
    print("TensorFlow not available. We'll use scikit-learn for image classification.")
    TF_AVAILABLE = False

# Built-in datasets
from sklearn.datasets import load_digits

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

# Set random seed
np.random.seed(42)

print("\nLibraries imported successfully!")
print("Ready to explore Computer Vision! 👁️🤖")

## 2. Understanding Digital Images

### 2.1 Image Representation and Basic Operations

In [ ]:
print("=== UNDERSTANDING DIGITAL IMAGES ===\n")

# Load the digits dataset for demonstration
digits = load_digits()
print(f"Digits dataset loaded:")
print(f"• Images: {digits.data.shape[0]}")
print(f"• Image size: {digits.images.shape[1]}x{digits.images.shape[2]} pixels")
print(f"• Classes: {len(np.unique(digits.target))} (digits 0-9)")
print(f"• Data shape: {digits.data.shape}")

# Visualize some sample images
fig, axes = plt.subplots(2, 5, figsize=(12, 6))
for i in range(10):
    row = i // 5
    col = i % 5
    
    # Display the image
    axes[row, col].imshow(digits.images[i], cmap='gray')
    axes[row, col].set_title(f'Digit: {digits.target[i]}')
    axes[row, col].axis('off')

plt.suptitle('Sample Handwritten Digits (8x8 pixels)', fontsize=14)
plt.tight_layout()
plt.show()

# Explore image properties
print(f"\nImage Analysis:")
sample_image = digits.images[0]
print(f"• Sample image shape: {sample_image.shape}")
print(f"• Data type: {sample_image.dtype}")
print(f"• Pixel value range: {sample_image.min():.1f} to {sample_image.max():.1f}")
print(f"• Mean pixel value: {sample_image.mean():.2f}")

# Show pixel values for a small section
print(f"\nPixel values (top-left 4x4 corner):")
print(sample_image[:4, :4])

# Demonstrate how images are flattened for machine learning
print(f"\nFlattened representation:")
print(f"• Original shape: {sample_image.shape}")
print(f"• Flattened shape: {digits.data[0].shape}")
print(f"• First 10 pixel values: {digits.data[0][:10]}")

## 3. Basic Image Processing

### 3.1 Image Transformations and Filters

In [ ]:
print("=== BASIC IMAGE PROCESSING ===\n")

# Select a sample image for processing
sample_idx = 100
original_image = digits.images[sample_idx]
print(f"Processing digit: {digits.target[sample_idx]}")

# Define some basic image processing functions
def apply_gaussian_blur(image, sigma=1.0):
    """Apply Gaussian blur to an image"""
    from scipy.ndimage import gaussian_filter
    return gaussian_filter(image, sigma=sigma)

def apply_edge_detection(image):
    """Simple edge detection using gradient"""
    from scipy.ndimage import sobel
    # Sobel edge detection
    edge_x = sobel(image, axis=0)
    edge_y = sobel(image, axis=1)
    return np.sqrt(edge_x**2 + edge_y**2)

def adjust_contrast(image, factor=1.5):
    """Adjust image contrast"""
    mean_val = image.mean()
    return np.clip((image - mean_val) * factor + mean_val, 0, 16)

def apply_threshold(image, threshold=8.0):
    """Apply binary thresholding"""
    return (image > threshold).astype(float) * 16

# Apply different transformations
transformations = {
    'Original': original_image,
    'Gaussian Blur': apply_gaussian_blur(original_image, sigma=0.8),
    'Edge Detection': apply_edge_detection(original_image),
    'High Contrast': adjust_contrast(original_image, factor=2.0),
    'Binary Threshold': apply_threshold(original_image, threshold=6.0),
    'Inverted': 16 - original_image
}

# Visualize transformations
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for i, (name, image) in enumerate(transformations.items()):
    axes[i].imshow(image, cmap='gray')
    axes[i].set_title(f'{name}')
    axes[i].axis('off')
    
    # Add some statistics
    axes[i].text(0.02, 0.98, f'Min: {image.min():.1f}\nMax: {image.max():.1f}', 
                transform=axes[i].transAxes, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.suptitle(f'Image Processing Techniques - Digit {digits.target[sample_idx]}', fontsize=16)
plt.tight_layout()
plt.show()

print(f"Key Concepts:")
print(f"• Gaussian Blur: Reduces noise and detail")
print(f"• Edge Detection: Highlights boundaries and features")
print(f"• Contrast Adjustment: Enhances visual differences")
print(f"• Thresholding: Converts to binary (black/white)")
print(f"• Inversion: Flips pixel intensities")

## 4. Traditional Machine Learning for Images

### 4.1 Feature Extraction and Classification

In [ ]:
print("=== TRADITIONAL ML FOR IMAGE CLASSIFICATION ===\n")

# Prepare the data
X = digits.data  # Flattened pixel values
y = digits.target  # Digit labels

print(f"Dataset information:")
print(f"• Total images: {X.shape[0]}")
print(f"• Features per image: {X.shape[1]} (pixels)")
print(f"• Classes: {len(np.unique(y))}")
print(f"• Class distribution: {dict(zip(*np.unique(y, return_counts=True)))}")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nData split:")
print(f"• Training samples: {X_train.shape[0]}")
print(f"• Test samples: {X_test.shape[0]}")

# Train and compare different models
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=3)
}

results = []

print(f"\nTraining and evaluating models:")
print("=" * 40)

for name, model in models.items():
    # Train model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    results.append({
        'Model': name,
        'Accuracy': accuracy
    })
    
    print(f"{name:20}: {accuracy:.3f}")

# Create results DataFrame
results_df = pd.DataFrame(results)
print(f"\nModel Comparison:")
print(results_df.round(3))

# Find best model
best_model_idx = results_df['Accuracy'].idxmax()
best_model_name = results_df.loc[best_model_idx, 'Model']
best_accuracy = results_df.loc[best_model_idx, 'Accuracy']

print(f"\nBest performing model: {best_model_name} ({best_accuracy:.3f} accuracy)")

## 5. Dimensionality Reduction for Images

### 5.1 Principal Component Analysis (PCA) for Images

In [ ]:
print("=== PCA FOR IMAGE ANALYSIS ===\n")

# Apply PCA to the digit images
pca = PCA(n_components=32, random_state=42)  # Reduce from 64 to 32 dimensions
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

print(f"PCA Results:")
print(f"• Original dimensions: {X_train.shape[1]}")
print(f"• Reduced dimensions: {X_train_pca.shape[1]}")
print(f"• Variance explained: {pca.explained_variance_ratio_.sum():.3f} ({pca.explained_variance_ratio_.sum()*100:.1f}%)")
print(f"• Dimensionality reduction: {X_train.shape[1]/X_train_pca.shape[1]:.1f}x")

# Visualize the principal components (eigendigits)
fig, axes = plt.subplots(4, 8, figsize=(16, 8))
for i in range(32):
    row = i // 8
    col = i % 8
    
    # Reshape component back to image format
    component_image = pca.components_[i].reshape(8, 8)
    
    axes[row, col].imshow(component_image, cmap='coolwarm')
    axes[row, col].set_title(f'PC{i+1}\n({pca.explained_variance_ratio_[i]:.1%})', fontsize=8)
    axes[row, col].axis('off')

plt.suptitle('Principal Components (Eigendigits)', fontsize=14)
plt.tight_layout()
plt.show()

# Test classification performance with PCA
rf_pca = RandomForestClassifier(n_estimators=100, random_state=42)
rf_pca.fit(X_train_pca, y_train)
pred_pca = rf_pca.predict(X_test_pca)
acc_pca = accuracy_score(y_test, pred_pca)

print(f"\nClassification with PCA features:")
print(f"• Accuracy: {acc_pca:.3f}")
print(f"• Feature reduction: {X_train.shape[1]} → {X_train_pca.shape[1]} dimensions")
print(f"• Maintains good performance with 50% fewer features")

# Show reconstruction examples
print(f"\nImage Reconstruction with PCA:")

# Reconstruct some test images
n_samples = 5
sample_indices = [0, 50, 100, 150, 200]

fig, axes = plt.subplots(2, n_samples, figsize=(15, 6))

for i, idx in enumerate(sample_indices):
    # Original image
    original = X_test[idx].reshape(8, 8)
    axes[0, i].imshow(original, cmap='gray')
    axes[0, i].set_title(f'Original\nDigit: {y_test[idx]}')
    axes[0, i].axis('off')
    
    # Reconstructed image
    pca_coords = X_test_pca[idx]
    reconstructed = pca.inverse_transform(pca_coords.reshape(1, -1))
    reconstructed = reconstructed.reshape(8, 8)
    
    axes[1, i].imshow(reconstructed, cmap='gray')
    axes[1, i].set_title('PCA\nReconstructed')
    axes[1, i].axis('off')

plt.suptitle('Original vs PCA Reconstructed Images', fontsize=14)
plt.tight_layout()
plt.show()

print(f"\nPCA Insights:")
print(f"• First few components capture most important image features")
print(f"• Reconstruction preserves essential digit characteristics")
print(f"• Significant dimensionality reduction with minimal information loss")

## 6. Modern Computer Vision Concepts

### 6.1 Understanding CNNs and Deep Learning for Images

In [ ]:
print("=== MODERN COMPUTER VISION CONCEPTS ===\n")

if TF_AVAILABLE:
    print("Exploring CNNs with TensorFlow...")
    
    # Prepare data for CNN (needs to be reshaped)
    X_train_cnn = X_train.reshape(-1, 8, 8, 1).astype('float32') / 16.0  # Normalize
    X_test_cnn = X_test.reshape(-1, 8, 8, 1).astype('float32') / 16.0
    
    # Convert labels to categorical
    y_train_cnn = keras.utils.to_categorical(y_train, 10)
    y_test_cnn = keras.utils.to_categorical(y_test, 10)
    
    print(f"CNN Data shapes:")
    print(f"• X_train: {X_train_cnn.shape} (samples, height, width, channels)")
    print(f"• y_train: {y_train_cnn.shape} (samples, classes)")
    
    # Build a simple CNN
    model = keras.Sequential([
        # Convolutional layers
        layers.Conv2D(16, (3, 3), activation='relu', input_shape=(8, 8, 1)),
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(32, (3, 3), activation='relu'),
        
        # Flatten and dense layers
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print(f"\nCNN Architecture:")
    model.summary()
    
    # Train the CNN
    print(f"\nTraining CNN...")
    history = model.fit(
        X_train_cnn, y_train_cnn,
        epochs=10,
        batch_size=32,
        validation_split=0.2,
        verbose=1
    )
    
    # Evaluate CNN
    test_loss, test_accuracy = model.evaluate(X_test_cnn, y_test_cnn, verbose=0)
    print(f"\nCNN Results:")
    print(f"• Test accuracy: {test_accuracy:.3f}")
    print(f"• Comparison with best traditional ML: {test_accuracy:.3f} vs {best_accuracy:.3f}")
    
else:
    print("TensorFlow not available - explaining CNN concepts:")
    
print(f"\nKey CNN Concepts:")
print(f"=" * 30)
concepts = {
    "Convolutional Layers": {
        "purpose": "Detect local features using filters/kernels",
        "advantage": "Preserve spatial relationships in images"
    },
    "Pooling Layers": {
        "purpose": "Reduce spatial dimensions and computational cost",
        "advantage": "Provide translation invariance"
    },
    "Feature Maps": {
        "purpose": "Represent detected features at different layers",
        "advantage": "Build hierarchical feature representations"
    },
    "Parameter Sharing": {
        "purpose": "Use same filters across entire image",
        "advantage": "Reduce parameters and improve generalization"
    }
}

for concept, details in concepts.items():
    print(f"\n{concept}:")
    print(f"  Purpose: {details['purpose']}")
    print(f"  Advantage: {details['advantage']}")

print(f"\nWhy CNNs Excel at Computer Vision:")
print(f"• Automatic feature learning (no manual feature engineering)")
print(f"• Hierarchical feature detection (edges → shapes → objects)")
print(f"• Translation invariance (detect features anywhere in image)")
print(f"• Efficient parameter sharing reduces overfitting")
print(f"• Can handle variable input sizes and complex patterns")

## 7. Practice Exercises

Now it's your turn to practice what you've learned!

### Exercise 1: Image Processing Exploration

Experiment with different image processing techniques:

In [ ]:
# Your code here
# Hint: Try different threshold values, blur parameters, or create your own filters

# Sample solution:
def create_custom_filter(image, filter_type='sharpen'):
    from scipy.ndimage import convolve
    
    if filter_type == 'sharpen':
        kernel = np.array([[-1, -1, -1],
                          [-1,  9, -1],
                          [-1, -1, -1]])
    elif filter_type == 'emboss':
        kernel = np.array([[-2, -1,  0],
                          [-1,  1,  1],
                          [ 0,  1,  2]])
    else:
        kernel = np.array([[0, -1,  0],
                          [-1, 5, -1],
                          [0, -1,  0]])
    
    return convolve(image, kernel)

# Test custom filters
test_image = digits.images[50]
sharpened = create_custom_filter(test_image, 'sharpen')
embossed = create_custom_filter(test_image, 'emboss')

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].imshow(test_image, cmap='gray')
axes[0].set_title('Original')
axes[1].imshow(sharpened, cmap='gray')
axes[1].set_title('Sharpened')
axes[2].imshow(embossed, cmap='gray')
axes[2].set_title('Embossed')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

### Exercise 2: Feature Engineering for Images

Create your own image features and test their effectiveness:

In [ ]:
# Your code here
# Hint: Extract features like symmetry, density, or geometric properties

# Sample solution:
def extract_advanced_features(images, original_shape=(8, 8)):
    """Extract advanced image features"""
    features = []
    
    for img_flat in images:
        img_2d = img_flat.reshape(original_shape)
        
        feature_vector = [
            # Intensity features
            img_flat.mean(),
            img_flat.std(),
            img_flat.max() - img_flat.min(),
            
            # Geometric features
            np.sum(img_2d > img_2d.mean()) / img_2d.size,  # Fill ratio
            
            # Edge features
            apply_edge_detection(img_2d).sum(),
            
            # Symmetry features
            np.corrcoef(img_2d.flatten(), np.fliplr(img_2d).flatten())[0,1],  # Horizontal symmetry
            np.corrcoef(img_2d.flatten(), np.flipud(img_2d).flatten())[0,1],  # Vertical symmetry
        ]
        
        features.append(feature_vector)
    
    return np.array(features)

# Test advanced features
X_train_advanced = extract_advanced_features(X_train)
X_test_advanced = extract_advanced_features(X_test)

# Train classifier with advanced features
rf_advanced = RandomForestClassifier(n_estimators=100, random_state=42)
rf_advanced.fit(X_train_advanced, y_train)
pred_advanced = rf_advanced.predict(X_test_advanced)
acc_advanced = accuracy_score(y_test, pred_advanced)

print(f"Advanced features accuracy: {acc_advanced:.3f}")
print(f"Feature importance: {rf_advanced.feature_importances_}")

## Summary

In this notebook, we've explored Computer Vision and Image Processing:

### Key Concepts Covered:
✅ **Digital Images**: Understanding pixel representations and image properties  
✅ **Image Processing**: Filters, transformations, and enhancement techniques  
✅ **Traditional ML**: Feature extraction and classification for images  
✅ **Dimensionality Reduction**: PCA for image compression and analysis  
✅ **Modern Computer Vision**: CNN concepts and deep learning approaches  

### Key Takeaways:
- Images are numerical arrays that can be processed mathematically
- Traditional image processing uses filters and transformations
- Machine learning can classify images using pixel features
- PCA effectively reduces image dimensionality while preserving information
- CNNs revolutionized computer vision through automatic feature learning
- Different approaches work better for different types of image problems

### Real-World Applications:
- **Medical Imaging**: Disease diagnosis from X-rays, MRIs
- **Autonomous Vehicles**: Object detection and navigation
- **Security Systems**: Face recognition and surveillance
- **Quality Control**: Defect detection in manufacturing
- **Social Media**: Photo tagging and content moderation

### Course Completion:
🎉 **Congratulations! You've completed the 10-week Data Science Course!** 🎉

You now have a comprehensive understanding of:
- Python programming for data science
- Data manipulation and visualization
- Statistical analysis and hypothesis testing
- Machine learning algorithms (supervised and unsupervised)
- Neural networks and deep learning
- Natural language processing
- Computer vision and image processing

### Homework Assignment
1. Complete all practice exercises above
2. Experiment with different image processing techniques
3. Compare traditional ML vs deep learning approaches
4. Build a complete image classification project

### Additional Resources
- [OpenCV Documentation](https://docs.opencv.org/)
- [Computer Vision: Algorithms and Applications](http://szeliski.org/Book/)
- [PyImageSearch Tutorials](https://pyimagesearch.com/)
- [TensorFlow Computer Vision Tutorials](https://www.tensorflow.org/tutorials/images)

**You are now ready to tackle real-world data science challenges!** 🚀👁️🤖